In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from scipy.optimize import minimize

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
# 로컬 버전

data_dir = Path('../input/dankook')
feature_dir = Path('../output/feature')
val_dir = Path('../output/oof_pred')
test_dir = Path('../output/test_pred')
sub_dir = Path('../output/sub')


train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

In [5]:
# # 코렙 

# from google.colab import drive
# drive.mount('/content/drive')

# data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
# feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
# val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
# test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
# sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [6]:
algorithm_name = 'adacv'
feature_name = 'stacking1'
model_name = f'{algorithm_name}_{feature_name}_2'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [7]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [8]:
def load_data(model_names, oof_list, test_list, feature_names=None,number_of_versions=None):
    if number_of_versions == None or number_of_versions == 1:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2']
    elif number_of_versions == 2:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver2.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver2.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',
                                  f'{model}_ver2_class0',f'{model}_ver2_class1',f'{model}_ver2_class2']
    elif number_of_versions == 2.1:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver3.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver3.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',
                                  f'{model}_ver3_class0',f'{model}_ver3_class1',f'{model}_ver3_class2']
    elif number_of_versions == 3:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver2.csv', delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver3.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver2.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver3.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',
                                  f'{model}_ver2_class0',f'{model}_ver2_class1', f'{model}_ver2_class2',
                                  f'{model}_ver3_class0', f'{model}_ver3_class1',f'{model}_ver3_class2']

In [9]:
all_oof = []
all_test = []
feature_names = []

model_names = ['lrcv_feature', 'etscv_feature', 'rfcv_feature', 'gbcv_feature','xgbcv_feature','lgbmcv_feature','adacv_feature','adacv_feature_2']
load_data(model_names, all_oof, all_test, feature_names,3)

model_names = ['lrcv_polynomial_feature','rfcv_polynomial_feature','etscv_polynomial_feature','gbcv_polynomial_feature','adacv_polynomial_feature','adacv_polynomial_feature_2']
load_data(model_names,all_oof, all_test,feature_names,3)

model_names = ['xgbcv_polynomial_feature','lgbmcv_polynomial_feature']
load_data(model_names,all_oof, all_test,feature_names,2.1)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)
all_oof.shape, all_test.shape, feature_names

((319923, 138),
 (80000, 138),
 ['lrcv_feature_ver1_class0',
  'lrcv_feature_ver1_class1',
  'lrcv_feature_ver1_class2',
  'lrcv_feature_ver2_class0',
  'lrcv_feature_ver2_class1',
  'lrcv_feature_ver2_class2',
  'lrcv_feature_ver3_class0',
  'lrcv_feature_ver3_class1',
  'lrcv_feature_ver3_class2',
  'etscv_feature_ver1_class0',
  'etscv_feature_ver1_class1',
  'etscv_feature_ver1_class2',
  'etscv_feature_ver2_class0',
  'etscv_feature_ver2_class1',
  'etscv_feature_ver2_class2',
  'etscv_feature_ver3_class0',
  'etscv_feature_ver3_class1',
  'etscv_feature_ver3_class2',
  'rfcv_feature_ver1_class0',
  'rfcv_feature_ver1_class1',
  'rfcv_feature_ver1_class2',
  'rfcv_feature_ver2_class0',
  'rfcv_feature_ver2_class1',
  'rfcv_feature_ver2_class2',
  'rfcv_feature_ver3_class0',
  'rfcv_feature_ver3_class1',
  'rfcv_feature_ver3_class2',
  'gbcv_feature_ver1_class0',
  'gbcv_feature_ver1_class1',
  'gbcv_feature_ver1_class2',
  'gbcv_feature_ver2_class0',
  'gbcv_feature_ver2_class1',


In [11]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [12]:
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features' : 'sqrt',
    'verbose' : 0,
    'random_state': 2020
}

rf_clf = RandomForestClassifier(**rf_params)

# Ada Boost Classifier parameters
ada_params = {
    'base_estimator': rf_clf,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'algorithm': 'SAMME.R',
    'random_state': 2020
}

In [13]:
mlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    print(f'Traning model for CV #{fold+1}')
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    ada_clf = AdaBoostClassifier(**ada_params)
    ada_clf.fit(X_train,y_train)

    stk_test_pred += ada_clf.predict_proba(all_test) / n_splits
    stk_oof_pred[val_idx] += ada_clf.predict_proba(X_val)

    mlogloss.append(log_loss(y_val,stk_oof_pred[val_idx]))
    print(f'{fold+1} logloss = {log_loss(y_val,stk_oof_pred[val_idx])}')

print('mean logloss= ', np.mean(mlogloss))

Traning model for CV #1
1 logloss = 0.1714212057802204
Traning model for CV #2
2 logloss = 0.16979062915291085
Traning model for CV #3
3 logloss = 0.16284484123956214
Traning model for CV #4
4 logloss = 0.16457147714420803
Traning model for CV #5
5 logloss = 0.15980324080758862
mean logloss=  0.165686278824898


# 제출 파일 및 기타 파일 생성

In [14]:
# submission 파일 생성

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(stk_test_pred, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [15]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [16]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')